In [6]:
import os
import mysql.connector as connector  # MySQL connector for database interaction
from mysql.connector import errorcode  # Import error codes for exception handling
import pandas as pd
from IPython.display import display  # For displaying DataFrame in Jupyter Notebook

# Load MySQL Credentials from Environment Variables
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
DB_HOST = os.getenv("DB_HOST", "localhost")  # Default to localhost if not specified

# Ensure credentials are provided
if not all([DB_USER, DB_PASSWORD, DB_NAME]):
    raise ValueError("⚠️ Missing database credentials. Ensure DB_USER, DB_PASSWORD, and DB_NAME are set.")

# Establish MySQL Connection
try:
    connection = connector.connect(
        user=DB_USER,  
        password=DB_PASSWORD,  
        database=DB_NAME,
        host=DB_HOST
    )
    print("\nConnected to MySQL database")
except connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Connection failed: Incorrect username or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(f"Error: {err}")
    exit()  # Exit script if connection fails

# Create cursor
cursor = connection.cursor()    

# Clear any unread results before executing a new query
while cursor.nextset():
    pass  

# **Overview of current business status query**
query = """
    SELECT 
        b.BookingID,
        b.BookingDate,
        c.FirstName AS Customer_FirstName,
        c.LastName AS Customer_LastName,
        t.TableNumber,
        b.Status AS BookingStatus,
        o.OrderID,
        o.OrderDate,
        o.TotalCost,
        s.FirstName AS Staff_FirstName,
        s.LastName AS Staff_LastName,
        d.Status AS DeliveryStatus
    FROM Bookings b
    LEFT JOIN Customer_Details c ON b.CustomerID = c.CustomerID
    LEFT JOIN Tables t ON b.TableID = t.TableID
    LEFT JOIN Orders o ON b.BookingID = o.BookingID
    LEFT JOIN Staff_Information s ON o.StaffID = s.StaffID
    LEFT JOIN Order_Delivery_Status d ON o.OrderID = d.OrderID
    WHERE b.BookingDate BETWEEN '2025-03-15' AND '2025-03-21'
    ORDER BY b.BookingDate ASC, o.OrderDate ASC;
"""

try:
    # Execute the query
    cursor.execute(query)

    # Fetch all records and convert to DataFrame
    df_results = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

    # Display the results
    print("\n📌 **Bookings & Orders View:**")
    display(df_results)

except connector.Error as err:
    print(f"Error executing query: {err}")


Connected to MySQL database

📌 **Bookings & Orders View:**


,BookingID,BookingDate,Customer_FirstName,Customer_LastName,TableNumber,BookingStatus,OrderID,OrderDate,TotalCost,Staff_FirstName,Staff_LastName,DeliveryStatus
0,146,2025-03-15,Jane,Doe,2,New Booking,153,2025-03-15,100.00,John,Doe,Delivered
1,147,2025-03-15,Bob,Smith,3,New Booking,154,2025-03-15,30.50,John,Doe,Delivered
2,148,2025-03-15,Alice,Johnson,4,New Booking,155,2025-03-15,38.00,John,Doe,Delivered
3,149,2025-03-15,Charlie,Brown,5,New Booking,156,2025-03-15,83.50,John,Doe,Delivered
4,150,2025-03-15,David,Taylor,6,New Booking,157,2025-03-15,27.50,John,Doe,Delivered
5,151,2025-03-15,Emily,White,7,New Booking,158,2025-03-15,40.00,John,Doe,Delivered
6,152,2025-03-15,Frank,Miller,8,New Booking,159,2025-03-15,47.50,John,Doe,Delivered
7,153,2025-03-15,Grace,Wilson,9,New Booking,160,2025-03-15,48.00,John,Doe,Delivered
8,154,2025-03-15,Hannah,Moore,10,New Booking,161,2025-03-15,31.50,John,Doe,Delivered
9,155,2025-03-15,John,Doe,11,New Booking,162,2025-03-15,25.00,John,Doe,Delivered


we are now going to update OrderID 194 as delivered using UpdateOrderStatus procedure

In [7]:
# Define the Order ID and new status
order_id_to_update = 194
new_status = "Delivered"
performed_by = "Alice Johnson"

try:
    # Call the stored procedure
    cursor.callproc("UpdateOrderStatus", (order_id_to_update, new_status, performed_by))
    
    # Commit the transaction
    connection.commit()
    
    print(f"Order ID {order_id_to_update} has been successfully updated to '{new_status}'.")

except connector.Error as err:
    print(f"Error updating order status: {err}")

# Verify the update
order_status_query = f"""
SELECT * FROM Order_Delivery_Status WHERE OrderID = {order_id_to_update};
"""
cursor.execute(order_status_query)
df_order_status = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

# Display updated order status
print("\n📌 Updated Order Status:")
display(df_order_status)

Order ID 194 has been successfully updated to 'Delivered'.

📌 Updated Order Status:


,DeliveryID,OrderID,Date,Status
0,48,194,2025-03-21,Delivered


In [8]:
# **Overview of current business status query**
query = """
    SELECT 
        b.BookingID,
        b.BookingDate,
        c.FirstName AS Customer_FirstName,
        c.LastName AS Customer_LastName,
        t.TableNumber,
        b.Status AS BookingStatus,
        o.OrderID,
        o.OrderDate,
        o.TotalCost,
        s.FirstName AS Staff_FirstName,
        s.LastName AS Staff_LastName,
        d.Status AS DeliveryStatus
    FROM Bookings b
    LEFT JOIN Customer_Details c ON b.CustomerID = c.CustomerID
    LEFT JOIN Tables t ON b.TableID = t.TableID
    LEFT JOIN Orders o ON b.BookingID = o.BookingID
    LEFT JOIN Staff_Information s ON o.StaffID = s.StaffID
    LEFT JOIN Order_Delivery_Status d ON o.OrderID = d.OrderID
    WHERE b.BookingDate BETWEEN '2025-03-15' AND '2025-03-21'
    ORDER BY b.BookingDate ASC, o.OrderDate ASC;
"""

try:
    # Execute the query
    cursor.execute(query)

    # Fetch all records and convert to DataFrame
    df_results = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

    # Display the results
    print("\n📌 **Bookings & Orders View:**")
    display(df_results)

except connector.Error as err:
    print(f"Error executing query: {err}")


📌 **Bookings & Orders View:**


,BookingID,BookingDate,Customer_FirstName,Customer_LastName,TableNumber,BookingStatus,OrderID,OrderDate,TotalCost,Staff_FirstName,Staff_LastName,DeliveryStatus
0,146,2025-03-15,Jane,Doe,2,New Booking,153,2025-03-15,100.00,John,Doe,Delivered
1,147,2025-03-15,Bob,Smith,3,New Booking,154,2025-03-15,30.50,John,Doe,Delivered
2,148,2025-03-15,Alice,Johnson,4,New Booking,155,2025-03-15,38.00,John,Doe,Delivered
3,149,2025-03-15,Charlie,Brown,5,New Booking,156,2025-03-15,83.50,John,Doe,Delivered
4,150,2025-03-15,David,Taylor,6,New Booking,157,2025-03-15,27.50,John,Doe,Delivered
5,151,2025-03-15,Emily,White,7,New Booking,158,2025-03-15,40.00,John,Doe,Delivered
6,152,2025-03-15,Frank,Miller,8,New Booking,159,2025-03-15,47.50,John,Doe,Delivered
7,153,2025-03-15,Grace,Wilson,9,New Booking,160,2025-03-15,48.00,John,Doe,Delivered
8,154,2025-03-15,Hannah,Moore,10,New Booking,161,2025-03-15,31.50,John,Doe,Delivered
9,155,2025-03-15,John,Doe,11,New Booking,162,2025-03-15,25.00,John,Doe,Delivered


In [9]:
# Close connection
cursor.close()
connection.close()
print("\n🔌 MySQL connection closed.")


🔌 MySQL connection closed.
